<h1>LAB 1 - TOÁN ỨNG DỤNG VÀ THỐNG KÊ - 18CLC<h1/>

Môn `MTH00051`: Cơ sở dữ liệu
Khoa CNTT – CTĐA, Trường ĐH KHTN – ĐHQG TP.HCM
`25/07/2020`

Bài tập cá nhân gồm một thành viên:
-   `18127231`: Đoàn Đình Toàn (GitHub: [@t3bol90](https://github.com/t3bol90))

---

## Cân bằng phản ứng hóa học (cộng điểm)

**Công thức hóa học** (chemical formula) của các phân tử hóa học có thể được mô tả trong Python bằng một danh sách ([`list`](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists)) các bộ ([`tuple`](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)). Chẳng hạn, công thức hóa học của nước $\text{H}_2\text{O}$ có thể được mô tả bởi danh sách `[("H", 2), ("O", 1)]`.

Viết hàm trả về chuỗi biểu diễn LaTeX của công thức hóa học.

In [2]:
def chemical_formula(mol):
    ans = ""
    for chem, bias in mol:
        if isinstance(chem, str):
            ans += f"\\text{{{chem}}}" + (f"_{{{bias}}}" if bias != 1 else "")
        elif isinstance(chem, list):
            if bias != 1:
                ans += "("
            ans += chemical_formula(chem)
            if bias != 1:
                ans += f")_{{{bias}}}"
    return ans


In [3]:
from IPython.display import Latex

mol = chemical_formula([("H", 2), ("O", 1)])
# sẽ được như
# mol = r"\text{H}_2\text{O}"
print(mol)
Latex("$" + mol + "$")

\text{H}_{2}\text{O}


<IPython.core.display.Latex object>

Mở rộng hàm trên để cho phép công thức có chứa các nhóm nguyên tử. Chẳng hạn, công thức hóa học của nhôm sunfat $\text{Al}_2(\text{S}\text{O}_4)_3$ có thể được mô tả bởi danh sách `[("Al", 2), ([("S", 1), ("O", 4)], 3)]`.

In [4]:
mol = chemical_formula([("Al", 2), ([("S", 1), ("O", 4)], 3)])
# sẽ được như
# mol = r"\text{Al}_2(\text{S}\text{O}_4)_3"
print(mol)
Latex("$" + mol + "$")

\text{Al}_{2}(\text{S}\text{O}_{4})_{3}


<IPython.core.display.Latex object>

Một **phản ứng hóa học** (chemical reaction) có thể được mô tả bởi 2 danh sách: danh sách các **phân tử tham gia phản ứng** (reactant) và danh sách các **phân tử sản phẩm** (product). Viết hàm cân bằng phản ứng hóa học và trả về chuỗi biểu diễn LaTeX của **phương trình hóa học** (chemical equation) đã cân bằng.

In [5]:
from IPython.display import Latex
from collections import Counter
import sympy

In [6]:
def counting_extractor(compound):
    """
    Return a dict of compound and it's counts,
    Eg: 'H2O' -> {'H': 2, 'O': 1}
    """
    extractor = {}

    def coex_helper(mol, mbias=1):
        nonlocal extractor
        for chem, bias in mol:
            if isinstance(chem, str):
                if chem not in extractor:
                    extractor[chem] = 0
                extractor[chem] += bias * mbias
            elif isinstance(chem, list):
                coex_helper(chem, bias)

    coex_helper(compound)
    return extractor

In [7]:
def balance_chemical_equation(lhs, rhs):
    lhs_cmp = {}
    rhs_cmp = {}
    lhs_listing_cmp = []
    rhs_listing_cmp = []
    for comp in lhs:
        comp_coex = counting_extractor(comp)
        lhs_cmp.update(Counter(comp_coex))
        lhs_listing_cmp.append(comp_coex)
    for comp in rhs:
        comp_coex = counting_extractor(comp)
        rhs_cmp.update(Counter(comp_coex))
        rhs_listing_cmp.append(comp_coex)

    w = len(lhs) + len(rhs)
    h = len(lhs_cmp)
    A = [[0] * w for _ in range(h)]
    variables = sorted(list(set(cmp[0] for cmp in lhs_cmp.items())))
    # print(variables)
    for col, compound in enumerate(lhs_listing_cmp):
        for row, comp in enumerate(variables):
            A[row][col] = compound[comp] if comp in compound else 0
    for col, compound in enumerate(rhs_listing_cmp):
        for row, comp in enumerate(variables):
            A[row][col + len(lhs)] = -compound[comp] if comp in compound else 0
    # print(A)
    A = sympy.Matrix(A)
    Q = A.nullspace()[0]
    Q *= sympy.lcm([x.q for x in Q])
    ans = ''
    for index, chem in enumerate(lhs):
        bias = str(Q[index]) if Q[index] != 1 else ""
        ans += bias + chemical_formula(chem) + (' + ' if (index != len(lhs) - 1) else '')
        ans += r" \to "
    for index, chem in enumerate(rhs):
        bias = str(Q[index + len(lhs)]) if Q[index + len(lhs)] != 1 else ""
        ans += bias + chemical_formula(chem) + (' + ' if (index != len(rhs) - 1) else '')
    return ans


In [8]:
H2O = [("H", 2), ("O", 1)]
H2 = [("H", 2)]
O2 = [("O", 2)]

equation = balance_chemical_equation([H2O], [H2, O2])
# sẽ được như
# equation = r"2\text{H}_2\text{O} \to 2\text{H}_2 + \text{O}_2"
print(equation)
Latex("$$" + equation + "$$")

2\text{H}_{2}\text{O} \to 2\text{H}_{2} + \text{O}_{2}


<IPython.core.display.Latex object>

In [9]:
Al_OH3 = [("Al", 1), ([("O", 1), ("H", 1)], 3)]
H2_SO4 = [("H", 2), ([("S", 1), ("O", 4)], 1)]
Al2_SO43 = [("Al", 2), ([("S", 1), ("O", 4)], 3)]
H2O = [("H", 2), ("O", 1)]
KMnO4 = [('K', 1), ('Mn', 1), ('O', 4)]
HCl = [('H', 1), ('Cl', 1)]
KCl = [('K', 1), ('Cl', 1)]
MnCl2 = [('Mn', 1), ('Cl', 2)]
H2O = [('H', 2), ('O', 1)]
Cl2 = [('Cl', 2)]

FeSO4 = [("Fe", 1), ("S",1),("O",4) ]
KMnO4 = [("K", 1), ("Mn",1), ("O", 4)]
H2SO4 = [("H", 2), ("S", 1), ("O", 4)] 
K2SO4 = [("K", 2), ("S", 1), ("O", 4)] 
MnSO4 = [("Mn", 1), ("S", 1), ("O", 4)] 
Fe2SO4_3 = [("Fe", 2), ([("S", 1), ("O", 4)],3)]
H2O = [("H", 2), ("O", 1)]

equation = balance_chemical_equation([FeSO4, KMnO4, H2SO4], [K2SO4, MnSO4, Fe2SO4_3,H2O])

Latex("$$" + equation + "$$")
# lhs = [KMnO4, HCl]
# rhs = [KCl, MnCl2, Cl2, H2O]
# equation = balance_chemical_equation(lhs, rhs)
# sẽ được như
# equation = r"2\text{Al}(\text{O}\text{H})_3 + 3\text{H}_2\text{S}\text{O}_4 \to \text{Al}_2(\text{S}\text{O}_4)_3 + 6\text{H}_2\text{O}"
print(equation)
Latex("$$" + equation + "$$")

10\text{Fe}\text{S}\text{O}_{4} +  \to 2\text{K}\text{Mn}\text{O}_{4} +  \to 8\text{H}_{2}\text{S}\text{O}_{4} \to \text{K}_{2}\text{S}\text{O}_{4} + 2\text{Mn}\text{S}\text{O}_{4} + 5\text{Fe}_{2}(\text{S}\text{O}_{4})_{3} + 8\text{H}_{2}\text{O}


<IPython.core.display.Latex object>

**Chúc mọi người một ngày vui vẻ.**